**Introduction** 
The following notebook contains code which was used to develop time series machine learning models for the forecasting of Mobile Network Operators traffic in Malawi. The models were trained on annomyzed call detail (CDR) data. Three machine learning models were developed which were: ARIMA, SARIMA and Exponential Smoothing. The ARIMA model emerged as the most accurate model based on Mean Absolute Error (MAE), Root Mean Absolute Error (RMSE) and Mean Squared Error (MSE). The models were developed using the Python programming language on a 2i2c cloud platform for research. 

**Data Preprocessing** - Import the necessary libraries for data preparation for model training and testing

In [ ]:
import pandas as pd
from datetime import datetime

Loading the data

In [ ]:
data = pd.read_csv('data_copy.csv')

# Extract year
data['year'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.year

# Extract month
data['month'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.month_name()  # Month name (e.g., January)

# Extract day of month
data['day_of_month'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.day

# Extract day of week 
data['day_of_week'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.strftime('%A')  # Day of week (e.g., Monday)

# You can add similar logic to extract other features like quarter or week of year (if needed)
data = data.sort_values(by='period_id')
# Select rows where 'teleservice_cat' is 'VOICE'
voice_data = data[data['teleservice_cat'] == 'VOICE']
voice_data_not_transformed = voice_data.ffill()

**Outlier Detection** - z-score

In [ ]:
# Calculate z-scores
z_scores = (voice_data - voice_data.mean()) / voice_data.std()
# Identify outliers based on z-score threshold (e.g., +/- 3)
outliers = voice_data[abs(z_scores) > 3]

# Replace outliers with the mean
handled_outliers = voice_data.where(~(abs(z_scores) > 3), voice_data.mean())

model_data = np.log1p(voice_data+ 1)


**One Hot Encoding** Preparing the data further for model training and testing. This started with importing the required libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import OneHotEncoder

One Hot Encoding the the selected features for model training 

In [ ]:
data = pd.read_csv('data_copy.csv')

# Extract year, month, day of month, and day of week
data['year'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.year
data['month'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.month_name()
data['day_of_month'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.day
data['day_of_week'] = pd.to_datetime(data['period_id'], format='%Y%m%d').dt.strftime('%A')

data = data.sort_values(by='period_id')

# Select rows where 'teleservice_cat' is 'VOICE'
voice_data = data[data['teleservice_cat'] == 'VOICE']


categorical_columns = ["day_of_week"]

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore')  # 'ignore' handles unseen categories

# Fit the encoder on the categorical data (learns the categories)
encoder.fit(voice_data[categorical_columns])

# Transform the data using the fitted encoder (creates one-hot encoded columns)
encoded_data = pd.DataFrame(encoder.transform(voice_data[categorical_columns]))


# Combine the encoded data with the rest of the data
voice_data = pd.concat([voice_data, encoded_data], axis=1)

voice_data = voice_data.dropna()


voice_data['day_of_week'] = voice_data['day_of_week'].astype('category')

categorical_columns = ["day_of_week"]
encoder = OneHotEncoder(handle_unknown='ignore')  # 'ignore' handles unseen categories

# Fit the encoder on the categorical data (learns the categories)
encoder.fit(voice_data[categorical_columns])

# Transform the data using the fitted encoder (creates one-hot encoded columns)
encoded_data = pd.DataFrame(encoder.transform(voice_data[categorical_columns]))

**Statistical Tests and Analysis** Time Series Plot and Dickey-Fuller Test

In [ ]:
# Create time series plot
plt.figure(figsize=(7, 5))
plt.plot(time_steps, filled_data, marker='o', linestyle='-')
plt.xlabel('Time Steps')
plt.ylabel('Data Values')
plt.title('Time Series Plot')
plt.grid(True)
plt.show()

# Perform Dickey-Fuller test
adf_result = adfuller(filled_data)

# Print test statistics
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
  print('\t{}: {}'.format(key, value))

**Statistical Tests and Analysis** Partial Autocorrelation Function (PACF) and Autocorrelation Function (ACF)

In [ ]:
# Calculate ACF and PACF with confidence intervals
lags = range(20)
confint_alpha = 0.05  # Confidence level (e.g., 95% confidence interval)

acf_values, confint_acf = acf(filled_data, nlags=19, alpha=confint_alpha)
pacf_values, confint_pacf = pacf(filled_data, nlags=19, alpha=confint_alpha)

# Plot ACF on ax1
ax1.plot(lags, acf_values)
ax1.fill_between(lags, confint_acf[:, 0], confint_acf[:, 1], color='black', alpha=0.9, label='Confidence Interval-ACF')
ax1.set_xlabel('Lags')
ax1.set_ylabel('Autocorrelation')
ax1.set_title('Autocorrelation Function (ACF)')
ax1.axhline(y=0, color='r', linestyle='--', label='Significance Level-ACF')  # Optional significance line

# Plot PACF on ax2
ax2.plot(lags, pacf_values)
ax2.fill_between(lags, confint_pacf[:, 0], confint_pacf[:, 1], color='orange', alpha=0.9, label='Confidence Interval-PACF')
ax2.set_xlabel('Lags')
ax2.set_ylabel('Partial Autocorrelation')
ax2.set_title('Partial Autocorrelation Function (PACF)')
ax2.axhline(y=0, color='black', linestyle='--', label='Significance Level-PACF')  # Optional significance line

# Add legends and display the plot
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
plt.tight_layout()
plt.show()

**Exponential Smoothing** We start with code for the exponential smoothing algorithm begining with importing the necessary machine learning python libraries

In [ ]:
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

**Function Definition for the model:** We defined all the necessary functions for the training and evaluation of the the model 

# Define parameter grid for alpha
param_grid = {'alpha': np.linspace(0.05, 0.95, 19)}  

# Define scoring metric
def mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)


def mae(y_true, y_pred):  # Define mae function 
  return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred)) 

best_model = None
best_score = np.inf
# Define time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=5)  


**Grid Search** Perform the Grid Search to find the optimal hyperparameter combination for the model

In [ ]:
# Grid search loop
for params in param_grid.values():
    # Perform cross-validation
                cv_scores = []
                cv_scores2 = []
                cv_scores3 = []
                for train_index, test_index in tscv.split(filled_data):
                    train, validation = filled_data.iloc[train_index], filled_data.iloc[test_index]
                    model = ExponentialSmoothing(filled_data, trend=None, seasonal=None, seasonal_periods=None)
                    alpha_value = params  # Extract the current alpha value from params
                    model_fit = model.fit(smoothing_level=0.05)
                    predictions = model_fit.forecast(steps=len(validation))
                    rmse = np.sqrt(mean_squared_error(validation, predictions))
                    mae = mean_absolute_error(validation, predictions)
                    mse = mean_squared_error(validation, predictions)
                    absolute_errors = np.abs(filled_data - predictions)
                    absolute_errors_numeric = pd.to_numeric(absolute_errors, errors='coerce')
                   

                    print("RMSE")
                    print(rmse)
                    print("MAE")
                    print(mae)
                    print("MSE")
                    print(mse)
                    #print(mape)

                    cv_scores.append(rmse)
                    cv_scores2.append(mae)
                    cv_scores3.append(mse)

                    print(f"Final RMSE: {min(cv_scores)}")
                    print(f"Final MAE: {min(cv_scores2)}")
                    print(f"Final MSE: {min(cv_scores3)}")
                    
                mean_cv_score = np.mean(cv_scores)
                #if mean_cv_score < best_score:
                best_score, best_model = mean_cv_score, model


Code for the visuals for the final results from the Grid Search

In [ ]:
plt.scatter(validation, predictions)
plt.plot(validation[-20:], validation[-20:], color="red", linestyle="--")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs. Actual Values")
plt.show()

**SARIMA** The following is the code for the Seasonal Autoregressive Moving Average Model, starting with the import of the necessary libraries

In [ ]:
from scipy import stats
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX

**SARIMA Model**: The data is split into train-test splits and the hyperparameters are initialized based on statistical results from data preprocessing

# Define hyperparameter ranges
p_values = [3]  # Adjust as needed
d_values = [0, 1]  # Adjust as needed
q_values = [0, 1, 2]  # Adjust as needed

# Create time series cross-validation object
tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as needed

# Initialize variables for best model tracking
best_score = float('inf')
best_model = None
best_order = None

# Perform hyperparameter tuning and cross-validation with exogenous variables
for p in p_values:
    for d in d_values:
        for q in q_values:
            order = (p, d, q)
            seasonal_order = (0, 0, 1, 7)  # No seasonality assumed here
            try:
                cv_scores = []
                cv_scores2 = []
                cv_scores3 = []

                for train_index, test_index in tscv.split(voice_data_copy):
                    train, validation = voice_data_copy.iloc[train_index], voice_data_copy.iloc[test_index]


                
                    # Ensure numerical dtypes
                    #train['total_duration'] = train['total_duration'].astype('float64')  # Adjust dtype as needed
                    
                    # Correct exog input
                    exog_train = train[['day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 
                                        'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 
                                        'day_of_week_Wednesday', 'month_April', 'month_August','month_December', 
                                        'month_February', 'month_January', 'month_July', 'month_June', 'month_March', 
                                        'month_May', 'month_November', 'month_October', 'month_September']]
                    exog_validate = validation[['day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 
                                        'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 
                                        'day_of_week_Wednesday', 'month_April', 'month_August','month_December', 
                                        'month_February', 'month_January', 'month_July', 'month_June', 'month_March', 
                                        'month_May', 'month_November', 'month_October', 'month_September']]
                    
                    # Select columns for training/validation
                    #training = np.asarray(train[['total_volume']])  # Convert to NumPy array
                    training = train[['volume_traffic']].to_numpy()  # Select and convert to NumPy array
                    validation_data = validation[['volume_traffic']].to_numpy()
                    training2 = train['volume_traffic']

                    training_date_data = train.copy()

                    training_date_data['period_id'] = pd.to_datetime(training_date_data['period_id'].astype(str)[:8], format='mixed', errors='coerce')



                    #numeric_training = np.asarray(training, dtype=float)  # Convert to float64 NumPy array
                    #feature_index = 2 
                    #univariate_series = training[:, feature_index]
                    #training = np.asarray(train[['total_volume']])
                    training_exog = np.asarray(exog_train)
                    validation_exog = np.asarray(exog_validate)
                    #validation_y = np.asarray(validation[['volume_traffic']])
                    #forecast_horizon = len(exog_validate)
                    
                    # Include exogenous variables
                    #model_with_exog = ARIMA(x_train, order=order, exog=exog_train)
                    #model_fit_exog = model_with_exog.fit()


                    # Include exogenous variables in ARIMA model
                    #model_with_exog = ARIMA(first_half, order=order, exog=training_exog)
                    # Include exogenous variables
                    model_with_exog = SARIMAX(training, order=order, exog=exog_train,
                                  seasonal_order=seasonal_order)
                    model_fit_exog = model_with_exog.fit()

                    stepping = len(validation)

                    # Generate predictions
                    predictions = model_fit_exog.forecast(steps=stepping, exog=validation_exog, alpha=0.05)
                    #conf = model_fit_exog.forecast(steps=stepping, exog=validation_exog, alpha=0.05) # 95% conf
                    
                    

                    validation_data = validation_data.reshape(-1) 

                    errors = validation_data-predictions
                    data_length = len(validation)
                    time_series = pd.date_range(start='2019-01-01', periods=data_length, freq='D')  # Monthly data for a year
                    
                    #actual_values = pd.Series([10, 12, 15, 18, 21, 23, 22, 20, 18, 16, 14, 12])
                    #predictions = pd.Series([11, 13, 14, 17, 20, 22, 21, 19, 17, 15, 13, 11])


                    mae = np.mean(np.abs(errors))  # Mean Absolute Error
                    mse = np.mean(errors**2)  # Mean Squared Error
                    rmse = np.sqrt(mse)  # Root Mean Squared Error
                    prediction_series = pd.Series(predictions)
                    validation_series = pd.Series(validation_data)

                    training_one_dim = training.reshape(-1) 
                    training_series = pd.Series(training_one_dim)
                    print(type(prediction_series))
                    print(type(validation_series))
                    print(f"MAE: {mae:.4f}")
                    print(f"MSE: {mse:.4f}")
                    print(f"RMSE: {rmse:.4f}")

                    # Get residuals
                    residuals = model_fit_exog.resid
                    
                    # Get predicted values
                    predicted_values = model_fit_exog.predict(dynamic=False)

Code for the visuals for the final model performance results

In [ ]:
  display(training_date_data)
                    # Plot residuals vs. predicted values
                    plt.scatter(predicted_values, residuals)
                    plt.xlabel("Predicted Values")
                    plt.ylabel("Residuals")
                    plt.title("Residuals vs. Predicted Values")
                    plt.show()
                    # Plot ACF of residuals
                    fig, ax1 = plt.subplots()
                    acf_values = acf(residuals, nlags=40)  # 
                    plt.plot(acf_values, marker='o', linestyle='-', alpha=0.7)
                    plt.axhline(y=1.96/np.sqrt(len(residuals)), color='r', linestyle='--', label='Upper Bound')
                    plt.axhline(y=-1.96/np.sqrt(len(residuals)), color='r', linestyle='--', label='Lower Bound')
                    plt.xlabel("Lags")
                    plt.ylabel("Autocorrelation")
                    plt.title("Autocorrelation Function (ACF) of Residuals")
                    plt.legend()
                    plt.tight_layout()
                    plt.show()

                    # Plot PACF of residuals
                    fig, ax2 = plt.subplots()
                    pacf_values = pacf(residuals, nlags=40)  # 
                    plt.plot(pacf_values, marker='o', linestyle='-', alpha=0.7)
                    plt.axhline(y=1.96/np.sqrt(len(residuals)), color='r', linestyle='--', label='Upper Bound')
                    plt.axhline(y=-1.96/np.sqrt(len(residuals)), color='r', linestyle='--', label='Lower Bound')
                    plt.xlabel("Lags")
                    plt.ylabel("Partial Autocorrelation")
                    plt.title("Partial Autocorrelation Function (PACF) of Residuals")
                    plt.legend()
                    plt.tight_layout()
                    plt.show()

                    time_index = pd.date_range(start='2019-04-01', end='2019-04-07', freq='D')  
                    prediction_plot = prediction_series.head(7)
                    validation_plot = validation_series.head(7)
                    training_plot = training_series.head(7)
                    prediction_plot.index = time_index
                    validation_plot.index = time_index
            

                    filtered_predictions = prediction_plot['2019-04-01':'2019-04-30']
                    filtered_validation = validation_plot['2019-04-01':'2019-04-30']

                    plt.plot(filtered_predictions, label='Predictions')
                    plt.plot(filtered_validation, label='Validation')
                    plt.title("Predictions for a 7 Day Period")
                    plt.tight_layout()
                    plt.legend()
                    plt.show()

**ARIMA** The following is the code for the Autoregressive Moving Average Model, starting with the import of the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from IPython.display import display
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import kpss
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf

**ARIMA Model**: The data is split into train-test splits and the hyperparameters are initialized based on statistical results from data preprocessing

In [ ]:
p = 1
d = 0
q = 0

# Define time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=5)  

# Define hyperparameter grid
p_values = [1, 2, 3]
d_values = [0, 1]
q_values = [1, 2]

p = float("inf")

# Perform grid search
best_score, best_model = float("inf"), None
best_order = None

for p in p_values:
    for d in d_values:
        for q in q_values:
            order = (p, d, q)
            try:
                # Perform cross-validation
                cv_scores = []
                cv_scores2 = []
                cv_scores3 = []
                for train_index, test_index in tscv.split(filled_data):
                    train, validation = filled_data.iloc[train_index], filled_data.iloc[test_index]
                    model = ARIMA(train, order=order)
                    model_fit = model.fit()
                    predictions = model_fit.forecast(steps=len(validation))
                    rmse = np.sqrt(mean_squared_error(validation, predictions))
                    mae = mean_absolute_error(validation, predictions)
                    mse = mean_squared_error(validation, predictions)
                    absolute_errors = np.abs(validation - predictions)
                    absolute_errors_numeric = pd.to_numeric(absolute_errors, errors='coerce')
                    mape = np.mean(np.abs(absolute_errors / validation)) * 100

                    mape = np.mean(np.abs((validation - predictions) / validation)) * 100  
                    print(f"Absolute Error Data Type: {absolute_errors.dtypes}")
                    print("RMSE")
                    print(rmse)
                    print("MAE")
                    print(mae)
                    print("MSE")
                    print(mse)
                

                    cv_scores.append(rmse)
                    cv_scores2.append(mae)
                    cv_scores3.append(mse)

                    print(f"Final RMSE: {min(cv_scores)}")
                    print(f"Final MAE: {min(cv_scores2)}")
                    print(f"Final MSE: {min(cv_scores3)}")
                    print(f"Final MAPE: {mape}")
                    print(f"MAPE data type: {mape.dtypes}")
                    print(f"Data Type: {filled_data.dtypes}")
                    print(f"Data Type - Predictions: {predictions.dtypes}")
                    print(f"Absolute Errors: {absolute_errors_numeric}")
                    print(f"Filled Data Null Count: {filled_data.isnull().sum()}")
                    print(f"Predictions Null : {predictions.isnull().sum()}")
                    print(f"Predictions: {predictions}")    

                mean_cv_score = np.mean(cv_scores)
                #if mean_cv_score < best_score:
                best_score, best_model = mean_cv_score, model
                best_order = order
            except Exception as e:
                print(f"Error fitting model: {e}")
                continue  # Handle potential model fitting errors

# Use best_model for final forecasting (trained on the entire dataset)
best_model = ARIMA(validation, order=order)
best_model_fit = best_model.fit()
predictions = best_model_fit.forecast(steps = len(validation))

rmse = np.sqrt(mean_squared_error(validation, predictions))
mae = mean_absolute_error(validation, predictions)
mape = np.mean(np.abs((validation - predictions) / validation)) * 100

print(f"Predictions Length: {len(predictions)}")
print(f"Validation Length: {len(validation)}")

predictions_limited = predictions[:len(validation)]  # Adjust slicing based on prediction size
print(f"Predictions Limited Length: {len(predictions_limited)}")

Code for the visuals for the final model performance results

In [ ]:
plt.scatter(validation, predictions)
plt.plot(validation[-20:], validation[-20:], color="red", linestyle="--")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs. Actual Values")
plt.show()